In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import load_dataset
from nltk.tokenize import word_tokenize
import re

Task
1. Creat a RAG pipeline that can take following text and answer following questions
2. Try different types of chunking to get better answers?
3. Does asking questions differently give better answers? Why?
4. Try a different similarity search instead of cosine similarity - do the answers improve?



In [3]:
sample_text = """
The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.

Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter. These tribes have unique languages, traditions, and knowledge of the ecosystem. However, many face threats from illegal land encroachment and industrial activities.

Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents. The Amazon River, which flows through the rainforest, is the second longest river in the world and carries more water than any other river.

Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection. Many organizations and governments are working to reduce illegal logging and promote reforestation initiatives.
"""

In [4]:
questions = [
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]

### 1. Creat a RAG pipeline that can take following text and answer following questions

In [5]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

In [ ]:
# Function to generate embeddings
def get_transformer_embeddings(texts):
    return model.encode(texts, convert_to_numpy=True)

# Function to retrieve relevant passage
def retrieve_passage(stored_texts, stored_embeddings, query):
    query_embedding = get_transformer_embeddings([query])

    similarities = cosine_similarity(query_embedding, stored_embeddings)[0]  # Ensure correct shape
    best_match_idx = np.argmax(similarities)
    return stored_texts[best_match_idx]

# Function to answer questions based on stored content
def answer_question(stored_texts, stored_embeddings, query):
    relevant_passage = retrieve_passage(stored_texts, stored_embeddings, query)
    return relevant_passage

In [7]:
def RAG_pipeline(stored_texts, stored_embeddings):
    
    print("\nSample Questions and Answers:\n")
    for question in questions:        
        response = answer_question(stored_texts, stored_embeddings, question)
        print(f"Q: {question}\nA: {response}\n")

### 2. Try different types of chunking to get better answers?

In [8]:
# Function to split text into meaningful chunks (paragraphs)
def split_text(text):
    return [para.strip() for para in re.split("\n+", text) if para.strip()]

# Store document embeddings
stored_texts = split_text(sample_text)  # Store each paragraph separately
stored_embeddings = model.encode(stored_texts, convert_to_numpy=True)  # Store embeddings for each chunk

In [39]:
stored_texts

['The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.',
 'Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.',
 'Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter. These tribes have unique languages, traditions, and knowledge of the ecosystem. However, many face threats from illegal land encroachment and industrial activities.',
 'Scientists believe that the Amazon plays a cruci

In [11]:
RAG_pipeline(stored_texts, stored_embeddings)


Sample Questions and Answers:

Q: What is the Amazon rainforest?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Which countries does the Amazon span across?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Why is deforestation a problem in the Amazon?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countr

In [46]:
#Alternative form of chunking

# Function to split text into meaningful chunks (sentences)
def split_text(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

# Store document embeddings
stored_texts = split_text(sample_text)  # Store each sentence separately
stored_embeddings = model.encode(stored_texts, convert_to_numpy=True)  # Store embeddings for each chunk

In [35]:
stored_texts

['The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.',
 'It spans across nine countries, including Brazil, Peru, and Colombia.',
 'The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.',
 'Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.',
 'This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.',
 'Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter.',
 'These tribes have unique languages, traditions, and knowledge of the ecosystem.',
 'However, many face threats from illegal land encroachment and industrial activities.',
 'Scientists believe that the 

In [47]:
RAG_pipeline(stored_texts, stored_embeddings)


Sample Questions and Answers:

Q: What is the Amazon rainforest?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.

Q: Which countries does the Amazon span across?
A: It spans across nine countries, including Brazil, Peru, and Colombia.

Q: Why is deforestation a problem in the Amazon?
A: Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.

Q: How does the Amazon rainforest affect global weather patterns?
A: Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents.

Q: What role do indigenous tribes play in the Amazon?
A: Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter.

Q: What is the 

In [ ]:
#Alternative form of chunking

# Function to split text into meaningful chunks (sentences)
def split_text(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

# Store document embeddings
stored_texts = split_text(sample_text)  # Store each sentence separately
stored_embeddings = model.encode(stored_texts, convert_to_numpy=True)  # Store embeddings for each chunk

In [ ]:
stored_texts

In [ ]:
RAG_pipeline(stored_texts, stored_embeddings)

### 4. Try a different similarity search instead of cosine similarity - do the answers improve?

In [ ]:
# try for one tf_idf, more, need to find what else

In [ ]:
def euclidean_distance(vec1, vec2):
    #Calculates the Euclidean distance between two vectors
    return np.linalg.norm(vec1 - vec2)

# Function to generate embeddings
def get_transformer_embeddings2(texts):
    return model.encode(texts, convert_to_numpy=True)

# Function to retrieve relevant passage
def retrieve_passage2(stored_texts, stored_embeddings, query):
    query_embedding = get_transformer_embeddings2([query])
    distances = [euclidean_distance(query_embedding, stored_vec) for stored_vec in stored_embeddings]
    best_match_idx = np.argmin(distances)

    return stored_texts[best_match_idx]

# Function to answer questions based on stored content
def answer_question2(stored_texts, stored_embeddings, query):
    relevant_passage = retrieve_passage2(stored_texts, stored_embeddings, query)
    return relevant_passage

In [ ]:
def tf_idf(_max_features=5000): # how would you use this?, send this INTO the cosine similarity
    vectorizer = TfidfVectorizer(max_features=_max_features) # max_features: Builds a vocabulary that only considers the top max_features ordered by term frequency across the corpus.
    X_train_tfidf = vectorizer.fit_transform(train_texts).toarray() # The optimal value depends on the size and characteristics of your dataset.
    X_test_tfidf = vectorizer.transform(test_texts).toarray()
    return X_train_tfidf, X_test_tfidf

# something like this example 

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(stored_texts)

query = "The sky is very blue."
query_vector = vectorizer.transform([query])

similarities = cosine_similarity(query_vector, tfidf_matrix)

# Get the most similar document
most_similar_index = similarities.argmax()
stored_texts[most_similar_index]